# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Event-Driven Architecture

**Subtopic** - Implementing a Simple Event-Driven System

**Use Case** - Develop a simple event-driven system in Python. This system should consist of an event dispatcher and multiple event handlers. The event dispatcher will generate random events and the event handlers will process these events. All events and their handlers should be defined within the system. Use the Python standard library for the implementation.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
```


# Prompt

Design an event processing pipeline for a message routing system. The system routes messages through different processors based on message types and processing rules. Each message must go through specific validation, transformation, and delivery stages while maintaining order and dependencies.

## Input Format

- A list of dictionaries representing messages:

```
[{
    "id": str,           # Unique message identifier
    "type": str,         # One of: "DATA", "CONTROL", "STATUS"
    "payload": dict,     # Message content and metadata
    "dependencies": list # List of message IDs that must be processed first
}]
```

## Output Format

- A list of dictionaries representing processed results:

```
[{
    "id": str,          # Original message ID
    "status": str,      # "SUCCESS", "FAILED", or "SKIPPED"
    "processors": list, # List of processor IDs that handled the message
    "result": dict     # Final transformed payload
}]
```

## Examples

Input:

```
messages = [
    {
        "id": "msg1",
        "type": "DATA",
        "payload": {"content": "hello", "checksum": "abc123"},
        "dependencies": []
    },
    {
        "id": "msg2",
        "type": "CONTROL",
        "payload": {"command": "validate", "target": "msg1"},
        "dependencies": ["msg1"]
    }
]
```

Output:

```
[
    {
        "id": "msg1",
        "status": "SUCCESS",
        "processors": ["validate_p1", "transform_p1", "route_p1"],
        "result": {"content": "HELLO", "validated": True}
    },
    {
        "id": "msg2",
        "status": "SUCCESS",
        "processors": ["control_p1", "route_p2"],
        "result": {"command_status": "completed"}
    }
]
```

# Requirements

## Explicit Requirements

1. Messages must be processed only after their dependencies
2. Each message type requires specific processor chain
3. All processors must validate input before processing
4. Failed processor should stop the chain for that message
5. System must maintain processing order for dependent messages
6. Must use event-driven approach.

## Implicit Requirements

1. Message IDs must be unique
2. Circular dependencies must be detected and rejected
3. Unknown message types must be marked as SKIPPED
4. Processors must not modify original input
5. System must track all processors that touched a message

## Function Signature

```python
def process_message_pipeline(messages: List[Dict]) -> List[Dict]
```

## Edge Cases

1. Skip processing if message references non-existent dependency
2. Mark as FAILED if message payload is missing required fields
3. Return empty list for empty input
4. Skip duplicate message IDs and keep first occurrence
5. Mark messages with circular dependencies as FAILED

## Constraints

1. Maximum 100 messages in input list
2. Message IDs must be alphanumeric and max 32 characters
3. Maximum 10 dependencies per message
4. Payload dictionary maximum depth of 3 levels
5. Processor chain maximum length of 5 processors
6. The solution should be contained in single file and must have driver function.
7. Implementation code must run on an online python compiler and must be testable.

In [ ]:
# code

"""Message processing pipeline using event-driven architecture."""

from typing import List, Dict, Set, Optional
from abc import ABC, abstractmethod
import re
import copy


class EventDispatcher:
    """Handles event subscriptions and notifies processors."""

    def __init__(self):
        """Initializes event dispatcher with observer dictionary."""
        self._observers = {}

    def subscribe(self, event_type: str, observer: 'MessageProcessor'):
        """Registers a processor for a specific event type."""
        if event_type not in self._observers:
            self._observers[event_type] = []
        self._observers[event_type].append(observer)

    def notify(self, event_type: str, message: Dict) -> (List[str], Dict):
        """Notifies processors subscribed to an event type."""
        if event_type not in self._observers:
            return [], message["payload"]
        processor_ids = []
        payload_copy = copy.deepcopy(message["payload"])
        for observer in self._observers[event_type]:
            result, payload_copy = observer.process(message, payload_copy)
            if result:
                processor_ids.append(observer.processor_id)
            else:
                break
        return processor_ids, payload_copy


class MessageProcessor(ABC):
    """Abstract base class for all message processors."""

    def __init__(self, processor_id: str):
        """Initializes processor with processor ID."""
        self.processor_id = processor_id

    @abstractmethod
    def validate(self, message: Dict, payload: Dict) -> bool:
        """Validates incoming message and payload."""
        pass

    @abstractmethod
    def process(self, message: Dict, payload: Dict) -> (bool, Dict):
        """Processes and optionally modifies payload."""
        pass


class RoutingProcessor(MessageProcessor):
    """Routes messages without changing them."""

    def validate(self, message: Dict, payload: Dict) -> bool:
        """Validates message for routing."""
        return True

    def process(self, message: Dict, payload: Dict) -> (bool, Dict):
        """Returns the payload."""
        return True, payload


class ValidationProcessor(MessageProcessor):
    """Validates required fields in the payload."""

    def validate(self, message: Dict, payload: Dict) -> bool:
        """Checks if required fields exist in payload."""
        required_fields = {"content", "checksum"}
        return all(field in payload for field in required_fields)

    def process(self, message: Dict, payload: Dict) -> (bool, Dict):
        """Marks payload as validated if valid."""
        if not self.validate(message, payload):
            return False, payload
        payload = payload.copy()
        payload["validated"] = True
        return True, payload


class TransformProcessor(MessageProcessor):
    """Transforms message content to uppercase."""

    def validate(self, message: Dict, payload: Dict) -> bool:
        """Checks if content is a string."""
        return isinstance(payload.get("content"), str)

    def process(self, message: Dict, payload: Dict) -> (bool, Dict):
        """Converts content to uppercase."""
        if not self.validate(message, payload):
            return False, payload
        payload = payload.copy()
        payload["content"] = payload["content"].upper()
        return True, payload


class ControlProcessor(MessageProcessor):
    """Processes control messages with commands."""

    def validate(self, message: Dict, payload: Dict) -> bool:
        """Checks for command and target in payload."""
        return all(k in payload for k in ["command", "target"])

    def process(self, message: Dict, payload: Dict) -> (bool, Dict):
        """Returns command status if valid."""
        if not self.validate(message, payload):
            return False, payload
        return True, {"command_status": "completed"}


class StatusProcessor(MessageProcessor):
    """Echoes status message payload."""

    def validate(self, message: Dict, payload: Dict) -> bool:
        """Checks if status message is valid."""
        return True

    def process(self, message: Dict, payload: Dict) -> (bool, Dict):
        """Returns a copy of the payload."""
        return True, payload.copy()


class MessageRouter:
    """Handles validation, routing, and processing of messages."""

    def __init__(self):
        """Initializes dispatcher, processors, and tracking fields."""
        self.dispatcher = EventDispatcher()
        self._setup_processors()
        self.processed_msgs: Set[str] = set()
        self.results: Dict[str, Dict] = {}

    def _setup_processors(self):
        """Subscribes processors to event types."""
        d = self.dispatcher
        d.subscribe("DATA", ValidationProcessor("validate_p1"))
        d.subscribe("DATA", TransformProcessor("transform_p1"))
        d.subscribe("DATA", RoutingProcessor("route_p1"))
        d.subscribe("CONTROL", ControlProcessor("control_p1"))
        d.subscribe("CONTROL", RoutingProcessor("route_p2"))
        d.subscribe("STATUS", StatusProcessor("status_p1"))

    def _validate_message(self, message: Dict, all_ids: Set[str]) -> Optional[str]:
        """Validates structure, ID, dependencies, and payload."""
        if not isinstance(message.get("id"), str):
            return "Invalid message ID"
        if not re.match("^[a-zA-Z0-9]{1,32}$", message["id"]):
            return "Invalid message ID format"
        if message["id"] in self.processed_msgs:
            return "Duplicate message ID"
        if not isinstance(message.get("dependencies"), list):
            return "Invalid dependencies format"
        if len(message["dependencies"]) > 10:
            return "Too many dependencies"
        for dep in message["dependencies"]:
            if dep not in all_ids:
                return "Non-existent dependency"

        def dict_depth(d, level=1):
            """Computes dictionary depth."""
            if not isinstance(d, dict) or not d:
                return level
            return max(
                dict_depth(v, level + 1) if isinstance(v, dict)
                else level + 1 for v in d.values()
            )

        if dict_depth(message.get("payload", {})) > 3:
            return "Payload too deep"
        return None

    def _has_circular_dependency(
        self, msg_id: str, deps: Set[str], visited: Set[str]
    ) -> bool:
        """Detects cycles in message dependencies."""
        if msg_id in visited:
            return True
        visited.add(msg_id)
        for dep_id in deps:
            if dep_id in self.results:
                dep_msg = self.results[dep_id]
                if self._has_circular_dependency(
                    dep_id, set(dep_msg["dependencies"]), visited
                ):
                    return True
        visited.remove(msg_id)
        return False

    def _can_process_message(self, message: Dict) -> bool:
        """Checks if dependencies are already processed."""
        return all(
            dep in self.processed_msgs for dep in message["dependencies"]
        )

    def _create_result(
        self,
        msg_id: str,
        status: str,
        processors: List[str] = None,
        result: Dict = None,
    ) -> Dict:
        """Constructs a result dictionary."""
        return {
            "id": msg_id,
            "status": status,
            "processors": processors or [],
            "result": result or {},
        }

    def process_messages(self, messages: List[Dict]) -> List[Dict]:
        """Processes messages via validation and dispatch."""
        if not messages or len(messages) > 100:
            return []
        seen = set()
        filtered = []
        for msg in messages:
            if msg["id"] not in seen:
                filtered.append(msg)
                seen.add(msg["id"])
        messages = filtered
        all_ids = set(msg["id"] for msg in messages)
        msg_status = {msg["id"]: None for msg in messages}
        msg_result = {msg["id"]: None for msg in messages}
        msg_processors = {msg["id"]: [] for msg in messages}
        msg_map = {msg["id"]: msg for msg in messages}
        processed = set()
        results = []
        pending = set(msg["id"] for msg in messages)
        while pending:
            progress = False
            to_remove = set()
            for mid in list(pending):
                msg = msg_map[mid]
                error = self._validate_message(msg, all_ids)
                if error == "Duplicate message ID":
                    to_remove.add(mid)
                    continue
                if error == "Non-existent dependency":
                    msg_status[mid] = "SKIPPED"
                    to_remove.add(mid)
                    continue
                if error:
                    msg_status[mid] = "FAILED"
                    to_remove.add(mid)
                    continue
                if self._has_circular_dependency(
                    mid, set(msg["dependencies"]), set()
                ):
                    msg_status[mid] = "FAILED"
                    to_remove.add(mid)
                    continue
                dep_statuses = [
                    msg_status.get(dep) for dep in msg["dependencies"]
                ]
                if any(s is None for s in dep_statuses):
                    continue
                if any(s in ("FAILED", "SKIPPED") for s in dep_statuses):
                    msg_status[mid] = "SKIPPED"
                    to_remove.add(mid)
                    continue
                if msg["type"] not in ["DATA", "CONTROL", "STATUS"]:
                    msg_status[mid] = "SKIPPED"
                    to_remove.add(mid)
                    continue
                processor_ids = []
                payload_copy = copy.deepcopy(msg["payload"])
                success = True
                failed_at = None
                for observer in self.dispatcher._observers[msg["type"]]:
                    result, payload_copy_new = observer.process(
                        msg, payload_copy
                    )
                    processor_ids.append(observer.processor_id)
                    if not result:
                        failed_at = len(processor_ids) - 1
                        success = False
                        break
                    payload_copy = payload_copy_new
                if not success and failed_at is not None:
                    msg_processors[mid] = processor_ids[:failed_at + 1]
                else:
                    msg_processors[mid] = processor_ids if success else []
                if success:
                    msg_status[mid] = "SUCCESS"
                    if msg["type"] == "DATA":
                        msg_result[mid] = {
                            k: v for k, v in payload_copy.items()
                            if k in ["content", "validated"]
                        }
                    else:
                        msg_result[mid] = payload_copy
                else:
                    msg_status[mid] = "FAILED"
                to_remove.add(mid)
                progress = True
            pending -= to_remove
            if not progress:
                for mid in pending:
                    msg_status[mid] = "FAILED"
                break
        for msg in messages:
            results.append(
                self._create_result(
                    msg["id"],
                    msg_status[msg["id"]] or "FAILED",
                    msg_processors[msg["id"]],
                    msg_result[msg["id"]],
                )
            )
        return results


def process_message_pipeline(messages: List[Dict]) -> List[Dict]:
    """Processes messages using router pipeline."""
    router = MessageRouter()
    return router.process_messages(messages)


if __name__ == "__main__":
    test_messages = [
        {
            "id": "msg1",
            "type": "DATA",
            "payload": {"content": "hello", "checksum": "abc123"},
            "dependencies": [],
        },
        {
            "id": "msg2",
            "type": "CONTROL",
            "payload": {"command": "validate", "target": "msg1"},
            "dependencies": ["msg1"],
        },
    ]
    results = process_message_pipeline(test_messages)
    for result in results:
        print(result)


In [ ]:
# tests

"""Minimal unit tests for process_message_pipeline."""

import unittest

from main import process_message_pipeline


class TestProcessMessagePipeline(unittest.TestCase):
    """Minimal tests for process_message_pipeline."""

    def test_success_and_chain_order(self):
        """Success and dependency chain order."""
        messages = [
            {
                "id": "A",
                "type": "DATA",
                "payload": {"content": "a", "checksum": "c"},
                "dependencies": [],
            },
            {
                "id": "B",
                "type": "DATA",
                "payload": {"content": "b", "checksum": "c"},
                "dependencies": ["A"],
            },
            {
                "id": "C",
                "type": "CONTROL",
                "payload": {"command": "go", "target": "B"},
                "dependencies": ["B"],
            },
        ]
        result = process_message_pipeline(messages)
        self.assertEqual([r["id"] for r in result], ["A", "B", "C"])
        self.assertTrue(all(r["status"] == "SUCCESS" for r in result))
        self.assertEqual(result[2]["result"], {"command_status": "completed"})

    def test_circular_dependency(self):
        """Circular dependencies are FAILED."""
        messages = [
            {"id": "A", "type": "DATA", "payload": {}, "dependencies": ["B"]},
            {"id": "B", "type": "DATA", "payload": {}, "dependencies": ["A"]},
            {"id": "C", "type": "DATA", "payload": {}, "dependencies": ["B"]},
        ]
        result = process_message_pipeline(messages)
        self.assertTrue(all(r["status"] == "FAILED" for r in result))

    def test_nonexistent_dependency(self):
        """Missing dependency is SKIPPED."""
        messages = [
            {
                "id": "A2",
                "type": "DATA",
                "payload": {"content": "abc", "checksum": "xyz"},
                "dependencies": ["NOPE"],
            }
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result[0]["status"], "SKIPPED")

    def test_dependency_on_failed_message(self):
        """Dependency on failed message is FAILED."""
        messages = [
            {"id": "A", "type": "DATA", "payload": {}, "dependencies": ["A"]},
            {
                "id": "B",
                "type": "CONTROL",
                "payload": {"command": "go", "target": "A"},
                "dependencies": ["A"],
            },
        ]
        result = process_message_pipeline(messages)
        msgA = next(r for r in result if r["id"] == "A")
        msgB = next(r for r in result if r["id"] == "B")
        self.assertEqual(msgA["status"], "FAILED")
        self.assertEqual(msgB["status"], "FAILED")

    def test_missing_payload_field(self):
        """DATA missing required fields is FAILED."""
        messages = [
            {
                "id": "M1",
                "type": "DATA",
                "payload": {"checksum": "xyz"},
                "dependencies": [],
            }
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result[0]["status"], "FAILED")

    def test_unknown_message_type(self):
        """Unknown type is SKIPPED."""
        messages = [
            {
                "id": "X1",
                "type": "UNKNOWN",
                "payload": {},
                "dependencies": []
            }
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result[0]["status"], "SKIPPED")

    def test_duplicate_message_ids(self):
        """Only first duplicate ID is processed."""
        messages = [
            {
                "id": "DUP",
                "type": "DATA",
                "payload": {"content": "a", "checksum": "b"},
                "dependencies": [],
            },
            {
                "id": "DUP",
                "type": "CONTROL",
                "payload": {"command": "c", "target": "DUP"},
                "dependencies": ["DUP"],
            },
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(len(result), 1)
        self.assertEqual(result[0]["id"], "DUP")
        self.assertEqual(result[0]["status"], "SUCCESS")

    def test_empty_input(self):
        """Empty input returns empty list."""
        self.assertEqual(process_message_pipeline([]), [])

    def test_complex_valid_dag(self):
        """Valid DAG of dependencies is SUCCESS."""
        messages = [
            {
                "id": "D",
                "type": "DATA",
                "payload": {"content": "d", "checksum": "dchk"},
                "dependencies": ["B", "C"],
            },
            {
                "id": "C",
                "type": "DATA",
                "payload": {"content": "c", "checksum": "cchk"},
                "dependencies": ["A"],
            },
            {
                "id": "B",
                "type": "DATA",
                "payload": {"content": "b", "checksum": "bchk"},
                "dependencies": ["A"],
            },
            {
                "id": "A",
                "type": "DATA",
                "payload": {"content": "a", "checksum": "achk"},
                "dependencies": [],
            },
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(len(result), 4)
        self.assertTrue(all(r["status"] == "SUCCESS" for r in result))

    def test_processor_chain_stops_on_failure(self):
        """Processor chain stops on failure."""
        messages = [
            {
                "id": "M1",
                "type": "DATA",
                "payload": {},
                "dependencies": []
            }
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result[0]["status"], "FAILED")
        self.assertEqual(len(result[0]["processors"]), 1)

    def test_basic_data_message(self):
        """Basic DATA message is SUCCESS."""
        messages = [
            {
                "id": "A1",
                "type": "DATA",
                "payload": {"content": "abc", "checksum": "xyz"},
                "dependencies": [],
            }
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result[0]["status"], "SUCCESS")
        self.assertEqual(result[0]["result"],
                         {"content": "ABC", "validated": True})

    def test_control_message_with_dependency(self):
        """CONTROL message with dependency is SUCCESS."""
        messages = [
            {
                "id": "A1",
                "type": "DATA",
                "payload": {"content": "abc", "checksum": "xyz"},
                "dependencies": [],
            },
            {
                "id": "B1",
                "type": "CONTROL",
                "payload": {"command": "run", "target": "A1"},
                "dependencies": ["A1"],
            },
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result[1]["status"], "SUCCESS")
        self.assertEqual(result[1]["result"], {"command_status": "completed"})

    def test_status_message(self):
        """STATUS message is SUCCESS."""
        messages = [
            {
                "id": "S1",
                "type": "STATUS",
                "payload": {"info": "ok"},
                "dependencies": []
            }
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result[0]["status"], "SUCCESS")
        self.assertEqual(result[0]["result"], {"info": "ok"})

    def test_processor_chain_stops_on_failure_2(self):
        """Processor chain stops on non-string content."""
        messages = [
            {
                "id": "F1",
                "type": "DATA",
                "payload": {"content": 123, "checksum": "xyz"},
                "dependencies": [],
            }
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result[0]["status"], "FAILED")
        self.assertEqual(len(result[0]["processors"]), 2)

    def test_max_boundary(self):
        """Max boundary of 100 messages is SUCCESS."""
        messages = [
            {
                "id": f"ID{i}",
                "type": "DATA",
                "payload": {"content": str(i), "checksum": "c"},
                "dependencies": [],
            }
            for i in range(100)
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(len(result), 100)
        self.assertTrue(all(r["status"] == "SUCCESS" for r in result))

    def test_over_max_boundary(self):
        """Over max boundary returns empty list."""
        messages = [
            {
                "id": f"ID{i}",
                "type": "DATA",
                "payload": {"content": str(i), "checksum": "c"},
                "dependencies": [],
            }
            for i in range(101)
        ]
        result = process_message_pipeline(messages)
        self.assertEqual(result, [])


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/73650/view

#### Model code:

```python
########################################################################
# Event-Driven Message Processing Pipeline
#
# This module defines a single function `process_message_pipeline(messages)`
# that processes a list of message dictionaries and returns a list of
# result dictionaries. It demonstrates an event-driven architecture with
# validation, transformation, routing, dependency management, and
# processor chaining.
#
# USAGE:
#   You can run this module as-is. It includes a simple driver function
#   at the bottom with example input. Adjust or replace the driver code
#   to run other tests.
#
#   The main entry point to use from another piece of code is:
#       results = process_message_pipeline(messages)
########################################################################

from typing import List, Dict, Set

# <Issue>: Model's implementation is not event-driven; processors are called
# directly, not via event subscriptions/notifications
def process_message_pipeline(messages: List[Dict]) -> List[Dict]:
    """
    Processes messages through a pipeline with validations,
    transformations, and routing, while respecting dependencies.

    Parameters:
    -----------
    messages : List[Dict]
        A list of dictionaries. Each dictionary must have:
        - "id": str           # Unique message identifier
        - "type": str         # One of: "DATA", "CONTROL", "STATUS"
        - "payload": dict     # Message content and metadata
        - "dependencies": list # List of message IDs that must be processed first

    Returns:
    --------
    List[Dict]
        A list of dictionaries representing processed results:
        - "id": str                # Original message ID
        - "status": "SUCCESS" | "FAILED" | "SKIPPED"
        - "processors": list       # List of processor names that handled the message
        - "result": dict           # Final transformed payload (empty if SKIPPED or FAILED)
    """

    ########################################################################
    # Edge case 3: Return empty list for empty input
    ########################################################################
    if not messages:
        return []

    ########################################################################
    # Configuration for required fields by message type.
    # If a message is missing any of these fields, it is marked FAILED.
    ########################################################################
    REQUIRED_FIELDS = {
        "DATA": ["content", "checksum"],
        "CONTROL": ["command", "target"],
        # For demonstration, assume "STATUS" requires "info".
        "STATUS": ["info"],
    }

    ########################################################################
    # Processor-chains for each message type. These are the names of the
    # "processors" that will handle the message. The actual functions that
    # do the processing are defined below.
    #
    # Note the maximum chain length ≤ 5 from the constraints.
    ########################################################################
    PROCESSOR_CHAINS = {
        "DATA":   ["validate_p1", "transform_p1", "route_p1"],
        "CONTROL": ["control_p1", "route_p2"],
        "STATUS": ["status_p1", "route_p3"],
    }

    ########################################################################
    # Helper: Check whether a message ID is valid (alphanumeric, ≤ 32 chars).
    ########################################################################
    def is_valid_message_id(msg_id: str) -> bool:
        if not msg_id:
            return False
        if len(msg_id) > 32:
            return False
        return msg_id.isalnum()

    ########################################################################
    # Step 1: Gather unique messages in the order they appear.
    #         - Skip duplicates (only keep first).
    #         - Skip invalid IDs.
    ########################################################################
    seen_ids = set()
    valid_messages = []
    for msg in messages:
        msg_id = msg.get("id")
        # If "id" is missing or invalid or a duplicate, skip this message.
        if not msg_id or not is_valid_message_id(msg_id):
            continue
        if msg_id in seen_ids:
            # Edge case 4: Skip duplicate message IDs, keep first occurrence
            continue
        seen_ids.add(msg_id)
        valid_messages.append(msg)

    # If no valid messages remain, return empty
    if not valid_messages:
        return []

    # Build a dictionary for quick lookup by ID
    messages_by_id = {m["id"]: m for m in valid_messages}

    ########################################################################
    # Step 2: Prepare base results for each message, defaulting to PENDING.
    #         We'll fill in status, processors, and result later.
    ########################################################################
    results_map = {
        m["id"]: {
            "id": m["id"],
            "status": "PENDING",
            "processors": [],
            "result": {}
        } for m in valid_messages
    }

    ########################################################################
    # Step 3: Build adjacency structures for dependency graph.
    #
    # We'll store edges as:
    #   graph[dep] -> list of messages that directly depend on 'dep'.
    #
    # Then we can do a topological sort or DFS-based approach to detect cycles.
    #
    # Meanwhile, handle "skip if references non-existent dependency".
    ########################################################################
    # <Issue>: Model makes import inside the code instead of at the top.
    from collections import defaultdict

    graph = defaultdict(list)
    indegree = defaultdict(int)

    # Initialize in-degree for all message IDs
    for m in valid_messages:
        indegree[m["id"]] = 0

    # Check dependencies for existence. If non-existent, we mark the message SKIPPED
    # immediately (and effectively remove it from the graph).
    for m in valid_messages:
        msg_id = m["id"]
        dependencies = m.get("dependencies", [])
        # If any dependency doesn't exist in messages_by_id, skip this message
        # as per edge case #1.
        for d in dependencies:
            if d not in messages_by_id:
                # Skip
                results_map[msg_id]["status"] = "SKIPPED"
                break
        else:
            # If we did not break, all dependencies exist. Build adjacency.
            for d in dependencies:
                graph[d].append(msg_id)
                indegree[msg_id] += 1

    ########################################################################
    # Step 4: Detect circular dependencies (and mark them FAILED).
    #         We'll do a DFS-based cycle detection. If a node is SKIPPED,
    #         we ignore it. If a cycle is found, all nodes in that cycle
    #         get marked as FAILED.
    #
    #         We'll do a standard DFS with color states:
    #            0 = unvisited, 1 = visiting, 2 = visited
    #
    #         If we detect a cycle, mark all nodes in the current recursion
    #         stack as FAILED.
    ########################################################################
    visited = {m["id"]: 0 for m in valid_messages}  # 0=unvisited
    recursion_stack = []

    def dfs_cycle_detect(node: str):
        if results_map[node]["status"] in ("SKIPPED", "FAILED"):
            return  # Already decided, skip
        if visited[node] == 1:
            # Found a cycle -> mark entire recursion stack as FAILED
            for n in recursion_stack:
                results_map[n]["status"] = "FAILED"
            return
        if visited[node] == 2:
            return

        visited[node] = 1
        recursion_stack.append(node)

        # Explore neighbors
        for nxt in graph[node]:
            # If neighbor not skipped/failed, DFS
            if results_map[nxt]["status"] not in ("SKIPPED", "FAILED"):
                dfs_cycle_detect(nxt)

        visited[node] = 2
        recursion_stack.pop()

    # Run DFS cycle detection on each unvisited node
    for m in valid_messages:
        mid = m["id"]
        if visited[mid] == 0:
            dfs_cycle_detect(mid)

    ########################################################################
    # Step 5: Perform a topological sort among the messages that are not
    #         SKIPPED or FAILED. We'll use Kahn's algorithm for clarity.
    ########################################################################
    # We'll only topologically sort messages with status=="PENDING".
    # Everyone else (SKIPPED/FAILED) won't be processed further.
    #
    # Because we have potentially removed some edges by skipping, let's
    # make a fresh queue of 0 in-degree nodes that are still PENDING.
    ########################################################################
    from collections import deque

    topo_order = []
    queue = deque()

    # Initialize queue with PENDING, in-degree=0
    for m in valid_messages:
        mid = m["id"]
        if results_map[mid]["status"] == "PENDING" and indegree[mid] == 0:
            queue.append(mid)

    while queue:
        cur = queue.popleft()
        topo_order.append(cur)
        # For each neighbor, reduce in-degree if neighbor is still PENDING
        for nxt in graph[cur]:
            if results_map[nxt]["status"] == "PENDING":
                indegree[nxt] -= 1
                if indegree[nxt] == 0:
                    queue.append(nxt)

    # Any PENDING message not in topo_order but still PENDING is unreachable
    # or part of a cycle that got only partially flagged.
    # Typically they'd remain if they had non-zero in-degree. If not in topo_order
    # and status is still PENDING, it means there's a cycle. Mark them FAILED.
    for m in valid_messages:
        mid = m["id"]
        if (
            results_map[mid]["status"] == "PENDING"
            and mid not in topo_order
        ):
            results_map[mid]["status"] = "FAILED"

    ########################################################################
    # Step 6: Define the actual processor functions. These must not modify
    #         the original message payload directly. Instead, they return
    #         an updated copy or partial result that can be merged.
    #
    #         All processors must do a simple validation check before
    #         proceeding, and if that fails, we raise an exception to
    #         indicate a failure that should stop the chain.
    ########################################################################
    
    def validate_data(msg: Dict) -> Dict:
        """
        For DATA type messages:
        - Ensures required fields are present.
        - Adds 'validated': True if successful.
        - Does not keep 'checksum' in final result per example.
        """
        payload = msg["payload"]
        # Double-check required fields
        for field in REQUIRED_FIELDS["DATA"]:
            if field not in payload:
                raise ValueError("Missing required DATA field.")
        # Return new partial result
        return {
            "content": payload["content"],
            "validated": True
        }

    def transform_data(msg: Dict, current_result: Dict) -> Dict:
        """
        Transforms content to uppercase.
        """
        # Basic validation
        if "content" not in current_result:
            raise ValueError("Missing content in partial result (DATA).")
        transformed = current_result["content"].upper()
        updated_result = dict(current_result)
        updated_result["content"] = transformed
        return updated_result

    def route_data(msg: Dict, current_result: Dict) -> Dict:
        """
        Example of routing step; does nothing significant in this example.
        """
        # Basic validation
        if "content" not in current_result:
            raise ValueError("Cannot route DATA without content.")
        return dict(current_result)

    def validate_control(msg: Dict) -> Dict:
        """
        For CONTROL type messages:
        - Ensures required fields are present.
        - We'll discard them in final output, but let's pass them along.
        """
        # <Issue>: Model doesn't check for payload depth validation (max 3
        # levels)
        payload = msg["payload"]
        
        for field in REQUIRED_FIELDS["CONTROL"]:
            if field not in payload:
                raise ValueError("Missing required CONTROL field.")
        # <Issue>: Model's CONTROL processor doesn't return correct result
        # format
        # Return an empty partial since example discards them
        return {}

    def apply_control(msg: Dict, current_result: Dict) -> Dict:
        """
        Applies some control logic.
        In the example, we ended up with {"command_status": "completed"}.
        We'll do that here.
        """
        updated_result = dict(current_result)
        updated_result["command_status"] = "completed"
        return updated_result

    def route_control(msg: Dict, current_result: Dict) -> Dict:
        """
        Routing for CONTROL, does nothing special in the example.
        """
        return dict(current_result)

    def validate_status(msg: Dict) -> Dict:
        """
        For STATUS type messages:
        - Ensures required fields are present.
        """
        payload = msg["payload"]
        for field in REQUIRED_FIELDS["STATUS"]:
            if field not in payload:
                raise ValueError("Missing required STATUS field.")
        return {"info": payload["info"]}

    def route_status(msg: Dict, current_result: Dict) -> Dict:
        """
        Example final step for STATUS messages.
        """
        return dict(current_result)

    # Create a mapping from processor name to actual callable
    PROCESSOR_FUNCTIONS = {
        "validate_p1": lambda msg, res=None: validate_data(msg),
        "transform_p1": lambda msg, res: transform_data(msg, res),
        "route_p1": lambda msg, res: route_data(msg, res),

        "control_p1": lambda msg, res=None: validate_control(msg),
        "route_p2": lambda msg, res: route_control(msg, res),

        "status_p1": lambda msg, res=None: validate_status(msg),
        "route_p3": lambda msg, res: route_status(msg, res),
    }

    ########################################################################
    # Step 7: Process each message in topological order if status==PENDING.
    #         - If unknown type, mark SKIPPED
    #         - If any processor fails, mark FAILED
    #         - Otherwise, STATUS=SUCCESS
    #
    #         NOTE: "Processor must not modify original input payload", so
    #         each step works on a copy.
    ########################################################################
    for mid in topo_order:
        if results_map[mid]["status"] != "PENDING":
            continue

        message = messages_by_id[mid]
        mtype = message.get("type", "")
        # If it's an unknown type, skip
        if mtype not in PROCESSOR_CHAINS:
            results_map[mid]["status"] = "SKIPPED"
            continue

        # <Issue>: Model doesn't check for processor chain length constraint
        # Build the chain
        chain = PROCESSOR_CHAINS[mtype]

        current_result = {}
        processors_list = []
        try:
            for processor_name in chain:
                processor_func = PROCESSOR_FUNCTIONS[processor_name]
                # Some processors expect res=None for their first step,
                # some expect res. We'll handle it carefully:
                if "validate" in processor_name or processor_name.endswith("p1"):
                    # If it's the first in chain, pass res=None if we have none
                    if current_result:
                        # we might pass it anyway if that is simpler
                        new_result = processor_func(message, current_result)
                    else:
                        new_result = processor_func(message)
                else:
                    # subsequent steps expect current_result
                    new_result = processor_func(message, current_result)
                current_result = new_result
                processors_list.append(processor_name)

            # Mark success
            results_map[mid]["status"] = "SUCCESS"
            results_map[mid]["processors"] = processors_list
            results_map[mid]["result"] = current_result

        except Exception:
            # On any failure in the chain, mark FAILED
            results_map[mid]["status"] = "FAILED"
            results_map[mid]["processors"] = processors_list
            results_map[mid]["result"] = {}
            # Stop processing further

    ########################################################################
    # Finally, build the output list in the original (valid) message order.
    ########################################################################
    output = []
    for m in valid_messages:
        mid = m["id"]
        output.append({
            "id": results_map[mid]["id"],
            "status": results_map[mid]["status"],
            "processors": results_map[mid]["processors"],
            "result": results_map[mid]["result"],
        })

    return output

########################################################################
# Driver code / Example test
########################################################################
if __name__ == "__main__":
    # Example input
    messages_input = [
        {
            "id": "msg1",
            "type": "DATA",
            "payload": {"content": "hello", "checksum": "abc123"},
            "dependencies": []
        },
        {
            "id": "msg2",
            "type": "CONTROL",
            "payload": {"command": "validate", "target": "msg1"},
            "dependencies": ["msg1"]
        }
    ]

    results = process_message_pipeline(messages_input)
    print("RESULTS:")
    for r in results:
        print(r)

    # Expected output (similar to the problem statement example):
    #
    # [
    #   {
    #       "id": "msg1",
    #       "status": "SUCCESS",
    #       "processors": ["validate_p1", "transform_p1", "route_p1"],
    #       "result": {"content": "HELLO", "validated": True}
    #   },
    #   {
    #       "id": "msg2",
    #       "status": "SUCCESS",
    #       "processors": ["control_p1", "route_p2"],
    #       "result": {"command_status": "completed"}
    #   }
    # ]
```